In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [13]:
STORM_NUMBER = 6
TITLE = "11-12 December 2020"
SUBTITLE = "8 AM 13 December 2020"
sts = utc(2020, 12, 12, 6)
ets = utc(2020, 12, 26, 23, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 30, 'z': 'yes', 'f': 'linear'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [6]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s, "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=0)
    return mp

In [25]:
cull = [358, 'CINI4', 102, 183, 'LSXI4', 174, 200, 'MTAI4', 'VNTI4', 618, 734, 718, 632, 551, 468, 'AESI4', 344, 215]
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["state"] == "MN"].index, USEME] = False
hardcode = [
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=0, upper=4)
res = mp.postprocess(filename='201213.png')
mp.close()

     state  wfo     val      lon      lat                             geo  \
299     NE  OAX  3.9000 -96.2400  41.2900  POINT (314175.324 -405286.993)   
301     NE  OAX  3.9000 -96.6300  40.7700  POINT (283890.014 -464451.320)   
303     IA  DVN  3.8000 -91.4100  41.9300  POINT (709248.404 -304167.420)   
304     IL  DVN  3.8000 -90.4600  41.4300  POINT (793521.450 -350810.336)   
305     NE  OAX  3.8000 -96.6900  40.7500  POINT (278924.717 -466877.017)   
...    ...  ...     ...      ...      ...                             ...   
1447    IA  DMX  0.0001 -93.2000  42.7500  POINT (554550.774 -227104.626)   
1448    MO  DVN  0.0001 -92.1667  40.4608  POINT (662006.546 -473190.125)   
1463    NE  FSD  0.0001 -96.9600  42.3800  POINT (249695.415 -286673.667)   
1516    IA  FSD  0.0001 -95.6110  42.5463  POINT (359418.267 -263183.403)   
1544    IL  ILX  0.0001 -90.3853  40.9464  POINT (805752.989 -403577.078)   

      used_for_analysis  nwsli  plotme source  
299                True    